# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
!pip install cartopy geoviews pyproj

  Obtaining dependency information for cartopy from https://files.pythonhosted.org/packages/41/f4/edfba7b0768930486d4f434183a500de75826aba82e6ffd4884ae2b410b9/Cartopy-0.22.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for geoviews from https://files.pythonhosted.org/packages/72/10/81aaca3edf7d7bfb14a9a17b5effaeb9dcba54ba9703b392bb17241e45ea/geoviews-1.11.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyproj from https://files.pythonhosted.org/packages/89/8f/27350c8fba71a37cd0d316f100fbd96bf139cc2b5ff1ab0dcbc7ac64010a/pyproj-3.6.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.7 from https://files.pythonhosted.org/packages/28/32/c404f6d5566025b9d94eef0cc21039b0a4c10a392bad2f81edf778b3c2bc/shapely-2.0.2-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
     ---------------------------------------- 46.5/46.5 kB 2.4 MB/s eta 0:00:00
   -----------

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID           City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0         kiunga  -6.1219  141.2906    306.62        47          60   
1        1        iqaluit  63.7506  -68.5145    237.00        61          20   
2        2           nuuk  64.1835  -51.7216    249.04        95          27   
3        3   east ballina -28.8667  153.5833    301.37        90         100   
4        4  blackmans bay -43.0167  147.3167    298.22        67          86   

   Wind Speed Country        Date  
0        1.96      PG  1706492130  
1        5.66      CA  1706492131  
2        4.75      GL  1706492040  
3        5.68      AU  1706492134  
4        3.63      AU  1706492135

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
import hvplot.pandas

# Plotting the map using hvplot.points
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    size='Humidity',
    geo=True,
    tiles='OSM',  # You can choose different tilesets (e.g., 'OSM', 'StamenTerrain', 'EsriImagery', etc.)
    frame_width=700,
    frame_height=500,
    scale=0.8,
    color="City"
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
city_data_df[city_data_df['Cloudiness'] == 0]

City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
5          5        anadyr  64.7500  177.4833    241.48        74           0   
6          6  port lincoln -34.7333  135.8667    295.57        56           0   
10        10     essaouira  31.5125   -9.7700    288.87        31           0   
17        17         ancud -41.8697  -73.8203    284.47        78           0   
18        18  longyearbyen  78.2186   15.6401    262.06        79           0   
..       ...           ...      ...       ...       ...       ...         ...   
535      535       mutuali -14.8706   37.0044    292.14        89           0   
545      545           gao  16.6362    1.6370    290.28        18           0   
555      555    las palmas  28.4204  -14.0131    288.59        42           0   
561      561        dakhla  23.6848  -15.9580    292.67        47           0   
565      565         pisco -13.7000  -76.2167    298.18        78           0   

     Wind Speed Country        Date  
5          4.00      RU  1706492135  
6          5.61      AU  1706492137  
10         2.06      MA  1706492140  
17         2.13      CL  1706492146  
18         5.14      SJ  1706492147  
..          ...     ...         ...  
535        1.08      MZ  1706492736  
545        5.12      ML  1706492745  
555        1.03      ES  1706492754  
561        3.41      EH  1706492760  
565        6.69      PE  1706492764  

[88 rows x 10 columns]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df[(city_data_df['Max Temp'] - 273 > 21) & (city_data_df['Max Temp'] - 273< 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]
# Drop any rows with null values
# YOUR CODE HERE
ideal_df = ideal_df.dropna()
# Display sample data
# YOUR CODE HERE
ideal_df.head()

City_ID            City      Lat       Lng  Max Temp  Humidity  \
139      139  cabo san lucas  22.8909 -109.9124    299.32        67   
194      194   mount gambier -37.8333  140.7667    297.66        47   
227      227  coronel oviedo -25.4167  -56.4500    295.18        58   
339      339      long beach  33.7670 -118.1892    296.47        44   
433      433            saki   8.6667    3.3833    297.03        20   

     Cloudiness  Wind Speed Country        Date  
139           0        2.06      MX  1706492284  
194           0        4.31      AU  1706492414  
227           0        1.79      PY  1706492447  
339           0        0.45      US  1706492296  
433           0        2.02      NG  1706492640

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.head()

City Country      Lat       Lng  Humidity Hotel Name
0         kiunga      PG  -6.1219  141.2906        47           
1        iqaluit      CA  63.7506  -68.5145        61           
2           nuuk      GL  64.1835  -51.7216        95           
3   east ballina      AU -28.8667  153.5833        90           
4  blackmans bay      AU -43.0167  147.3167        67

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
latitude = -33.8698439
longitude = 151.2082848

categories = "accommodation.hotel"

filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

limit = 20

params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City'].title()} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kiunga - nearest hotel: No hotel found
Iqaluit - nearest hotel: Frobisher Inn
Nuuk - nearest hotel: Nordbo
East Ballina - nearest hotel: Ballina Beach Resort
Blackmans Bay - nearest hotel: Villa Howden
Anadyr - nearest hotel: Гостевой дом
Port Lincoln - nearest hotel: Boston Hotel
Mahina - nearest hotel: Motu Martin
Port Macquarie - nearest hotel: The Observatory
San Luis De La Loma - nearest hotel: No hotel found
Essaouira - nearest hotel: Essaouira Wind Palace
Saudarkrokur - nearest hotel: Mikligarður
Albany - nearest hotel: No hotel found
Qaqortoq - nearest hotel: Hotel Qaqortoq
Governador Valadares - nearest hotel: Hotel Oitis
Washington - nearest hotel: Cashmere's Village Inn
Hawaiian Paradise Park - nearest hotel: No hotel found
Ancud - nearest hotel: Cabañas
Longyearbyen - nearest hotel: The Vault
Huarmey - nearest hotel: Hostal Santa Rosa
Thompson - nearest hotel: Thompson Inn
Constantia - nearest hotel: Hotel Chérica
Lakatoro - nearest hotel: No hotel fou

San Vicente - nearest hotel: Los Lapachos
Monjaras - nearest hotel: No hotel found
Preobrazheniye - nearest hotel: No hotel found
Fort Irwin - nearest hotel: Landmark Inn
Seaford - nearest hotel: Wellington Hotel
Mount Gambier - nearest hotel: The Old Mount Gambier Gaol
Mumbwa - nearest hotel: La Hacienda
Saipan - nearest hotel: Chalan Kanoa Beach Hotel
Carnarvon - nearest hotel: No hotel found
Alaghsas - nearest hotel: Hôtel Zagado
St. John'S - nearest hotel: Alt Hotel
Axams - nearest hotel: Kompetenzzentrum Lizum 1600
Jayapura - nearest hotel: Hotel Jasmine Jayapura
Oranjemund - nearest hotel: Op My Stoep
Najran - nearest hotel: No hotel found
Rikaze - nearest hotel: Gesar Hotel
Rongelap - nearest hotel: No hotel found
Tindouf - nearest hotel: محمد بوسبي
Zhezqazghan - nearest hotel: No hotel found
Anlu - nearest hotel: No hotel found
Enriquillo - nearest hotel: Hotel Juan José
Labrador City - nearest hotel: Carol Inn
Nemuro - nearest hotel: お宿エクハシ
Shenkursk - nearest hotel: No hotel 

Vanimo - nearest hotel: CBC Guesthouse
Wynyard - nearest hotel: Wynyard Hotel
Kuanda - nearest hotel: No hotel found
Tralee - nearest hotel: The Ashe Hotel
Ulety - nearest hotel: No hotel found
Narvik - nearest hotel: Narvik hotel Wivel
Rio Grande - nearest hotel: Hotel Vila Moura Executivo
Santa Cruz - nearest hotel: Aviador Hotel Boutique
San Jose Del Guaviare - nearest hotel: El Portico
Almyros - nearest hotel: ALOS
Katsuura - nearest hotel: 潮騒の宿
Gamba - nearest hotel: No hotel found
Ta`U - nearest hotel: No hotel found
Sola - nearest hotel: No hotel found
Shakawe - nearest hotel: EdenWood guesthouse
Inhambane - nearest hotel: Hotel de Inhambane
Mezhdurechensk - nearest hotel: Югус
Poggio Rusco - nearest hotel: No hotel found
Ghanzi - nearest hotel: Ghanzi farmhouse
Enewetak - nearest hotel: No hotel found
Muroto-Misakicho - nearest hotel: 民宿うらしま
Galveston - nearest hotel: Comfort Inn & Suites Texas City
Prabumulih - nearest hotel: No hotel found
Tomball - nearest hotel: La Quinta I

Varde - nearest hotel: Hotel Arnbjerg
Umluj - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity            Hotel Name
0           kiunga      PG  -6.1219  141.2906        47        No hotel found
1          iqaluit      CA  63.7506  -68.5145        61         Frobisher Inn
2             nuuk      GL  64.1835  -51.7216        95                Nordbo
3     east ballina      AU -28.8667  153.5833        90  Ballina Beach Resort
4    blackmans bay      AU -43.0167  147.3167        67          Villa Howden
..             ...     ...      ...       ...       ...                   ...
563           tefe      BR  -3.3542  -64.7114        99  Pousada Santa Teresa
564       el bauga      SD  18.2620   33.9081        33        No hotel found
565          pisco      PE -13.7000  -76.2167        78            La Portada
566          varde      DK  55.6211    8.4807        84        Hotel Arnbjerg
567          umluj      SA  25.0213   37.2685        60        No hotel found

[568 rows x 6 columns]

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

map_plot2 = hotel_df[hotel_df['Hotel Name'] != 'No hotel found'].hvplot.points(
    'Lng', 'Lat',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'],
    geo=True,
    tiles='OSM',  # You can choose different tilesets (e.g., 'OSM', 'StamenTerrain', 'EsriImagery', etc.)
    frame_width=700,
    frame_height=500,
    color='City',
    scale=0.5,
    size='Humidity'
)

# Display the plot
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)